In [1]:
import pandas as pd


In [ ]:
df = pd.read_csv("/Users/francis/Desktop/Github/elon-musk-sentiment-stock/data/musk_tweets_with_tsla_return.csv")
df = df.dropna(subset=["pct_change_next_day"])
df["target"] = (df["pct_change_next_day"] > 0).astype(int)


Index(['orig_tweet_id', 'orig_tweet_created_at', 'orig_tweet_text',
       'orig_tweet_url', 'orig_tweet_twitter_url', 'orig_tweet_username',
       'orig_tweet_retweet_count', 'orig_tweet_reply_count',
       'orig_tweet_like_count', 'orig_tweet_quote_count',
       'orig_tweet_view_count', 'orig_tweet_bookmark_count', 'musk_tweet_id',
       'musk_quote_tweet', 'musk_quote_retweet_count',
       'musk_quote_reply_count', 'musk_quote_like_count',
       'musk_quote_quote_count', 'musk_quote_view_count',
       'musk_quote_bookmark_count', 'musk_quote_created_at', 'vader_compund',
       'vader_label', 'vader_compound', 'tweet_date', 'musk_quote_date',
       'Date_', 'Close_TSLA', 'next_trading_day', 'next_day_close',
       'pct_change_next_day', 'target'],
      dtype='object')


In [5]:
features = ['vader_compound',
    'musk_quote_like_count',
    'musk_quote_retweet_count',
    'musk_quote_quote_count',
    'musk_quote_view_count']

In [7]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

/Users/francis/Desktop/Github/elon-musk-sentiment-stock/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [23:31:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.5311601150527325
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.62      0.59       563
           1       0.49      0.42      0.45       480

    accuracy                           0.53      1043
   macro avg       0.52      0.52      0.52      1043
weighted avg       0.53      0.53      0.53      1043

Confusion Matrix:
 [[351 212]
 [277 203]]


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load FinBERT
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# Create sentiment pipeline
finbert_pipeline = pipeline("sentiment-analysis", model=finbert_model, tokenizer=finbert_tokenizer)


/Users/francis/Desktop/Github/elon-musk-sentiment-stock/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use mps:0


In [11]:
def get_finbert_sentiment(text):
    try:
        result = finbert_pipeline(text[:512])  # max 512 tokens
        label = result[0]['label'].lower()     # 'positive', 'negative', 'neutral'
        score = result[0]['score']
        return pd.Series([label, score])
    except:
        return pd.Series([None, None])

# Apply to your dataset
df[['finbert_label', 'finbert_score']] = df['musk_quote_tweet'].apply(get_finbert_sentiment)


In [18]:
print(df[['musk_quote_tweet', 'finbert_label', 'finbert_score']].head(50))

                                     musk_quote_tweet finbert_label  \
0                                                 Yup       neutral   
1                         Massive public manipulation       neutral   
3                                                  🤣🤣       neutral   
4                                           Prescient       neutral   
6        Congratulations Tesla team on a great year!!      positive   
7                             Improved longform posts      positive   
8                                     Prelude to Mars       neutral   
9                Key milestone completed for flight 2       neutral   
10  “Ads” like this where you learn or are enterta...       neutral   
12              Great conversation with @NarendraModi       neutral   
14  Once again, I’d like to offer this platform to...       neutral   
15                                               Bold       neutral   
16                             Why ESG is the devil …       neutral   
17    

In [12]:
# Optional: Map label to numerical value
sentiment_map = {'positive': 1, 'neutral': 0, 'negative': -1}
df['finbert_label_num'] = df['finbert_label'].map(sentiment_map)


In [15]:
features = [
    'finbert_label_num',
    'musk_quote_like_count',
    'musk_quote_retweet_count',
    'musk_quote_quote_count',
    'musk_quote_view_count'
]


In [16]:
X = df[features]
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/Users/francis/Desktop/Github/elon-musk-sentiment-stock/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [23:56:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.5071907957813998
Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.61      0.57       563
           1       0.46      0.38      0.42       480

    accuracy                           0.51      1043
   macro avg       0.50      0.50      0.50      1043
weighted avg       0.50      0.51      0.50      1043

Confusion Matrix:
 [[345 218]
 [296 184]]


In [19]:
# Load Twitter-RoBERTa model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create a pipeline for sentiment
twitter_sentiment = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use mps:0
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [20]:
# Map the labels returned by Twitter-RoBERTa
def classify_roberta_sentiment(text):
    try:
        result = twitter_sentiment(text[:512])[0]  # BERT limit
        label_map = {'LABEL_0': 'negative', 'LABEL_1': 'neutral', 'LABEL_2': 'positive'}
        score_map = {'LABEL_0': -1, 'LABEL_1': 0, 'LABEL_2': 1}
        label = label_map[result['label']]
        score = score_map[result['label']] * result['score']  # Score weighted by direction
        return pd.Series([label, score])
    except:
        return pd.Series([None, None])

# Apply to your dataset
df[['roberta_label', 'roberta_score']] = df['musk_quote_tweet'].apply(classify_roberta_sentiment)


In [21]:
df['hybrid_sentiment'] = (
    0.5 * df['vader_compound'] + 
    0.5 * df['roberta_score']
)

features_roberta = [
    'roberta_score',
    'musk_quote_like_count',
    'musk_quote_retweet_count',
    'musk_quote_quote_count',
    'musk_quote_view_count'
]

features_hybrid = [
    'hybrid_sentiment',
    'musk_quote_like_count',
    'musk_quote_retweet_count',
    'musk_quote_quote_count',
    'musk_quote_view_count'
]

In [23]:
def run_xgboost_experiment(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    return model

X_roberta = df[features_roberta]
X_hybrid = df[features_hybrid]
y = (df['pct_change_next_day'] > 0).astype(int)

print("Roberta-only Model:")
model_roberta = run_xgboost_experiment(X_roberta, y)

print("\n Hybrid (VADER + Roberta) Model:")
model_hybrid = run_xgboost_experiment(X_hybrid, y)

Roberta-only Model:


/Users/francis/Desktop/Github/elon-musk-sentiment-stock/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:43:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.5455417066155321
              precision    recall  f1-score   support

           0       0.57      0.62      0.60       563
           1       0.51      0.46      0.48       480

    accuracy                           0.55      1043
   macro avg       0.54      0.54      0.54      1043
weighted avg       0.54      0.55      0.54      1043


 Hybrid (VADER + Roberta) Model:


/Users/francis/Desktop/Github/elon-musk-sentiment-stock/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:43:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.513902205177373
              precision    recall  f1-score   support

           0       0.54      0.62      0.58       563
           1       0.47      0.39      0.43       480

    accuracy                           0.51      1043
   macro avg       0.51      0.50      0.50      1043
weighted avg       0.51      0.51      0.51      1043



In [25]:
df["volatility_label"] = (df['pct_change_next_day'].abs() > 0.02).astype(int)
features = [
    'roberta_score',
    'vader_compound',
    'musk_quote_like_count',
    'musk_quote_retweet_count',
    'musk_quote_quote_count',
    'musk_quote_view_count'
]
X = df[features]
y = df['volatility_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Volatility Model Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/Users/francis/Desktop/Github/elon-musk-sentiment-stock/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:57:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Volatility Model Accuracy: 0.573346116970278
Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.28      0.33       397
           1       0.63      0.76      0.69       646

    accuracy                           0.57      1043
   macro avg       0.52      0.52      0.51      1043
weighted avg       0.55      0.57      0.55      1043

Confusion Matrix:
 [[110 287]
 [158 488]]
